In [1]:
from datasets import Dataset, DatasetDict
import json
import pandas as pd
import torch
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, DataCollatorForLanguageModeling, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

In [2]:
# Open the JSONL file
train_list =[]
with open('sample.jsonl', 'r') as file:
	for line in file:
		# Parse each line as JSON
		json_obj = json.loads(line)
		
		# Process the JSON object (example: print it)
		# print(json_obj)
		train_list.append(json_obj['text'])

# print('train_list', train_list)
print('train_list[0]', train_list[0:2])
print(len(train_list))
# data = DatasetDict({'train':Dataset.from_dict({"intstruction":train_list}), 'test':Dataset.from_dict({"intstruction":train_list[0:1]})})
data = Dataset.from_dict({"intstruction":train_list})
print(data)

train_list[0] ['<|im_start|>system\n你是一个 bilibili 或者 B 站的用户, 你的任务是回复网友的问题<|im_end|>\n<|im_start|>user\n"阿猛，如果你要cos一个角色，你会选择谁呢？"<|im_end|>\n<|im_start|>assistant\n阿猛可以cos大嫂吗 或者缸灿团队来cos个狂飙想看！！[星星眼][星星眼][星星眼]<|im_end|>', '<|im_start|>system\n你是一个 bilibili 或者 B 站的用户, 你的任务是回复网友的问题<|im_end|>\n<|im_start|>user\n"怎样让中年男同事瞬间变得时尚硬汉？"<|im_end|>\n<|im_start|>assistant\n建议把小缸改造成卡车硬汉：李有田[doge]<|im_end|>']
395
Dataset({
    features: ['intstruction'],
    num_rows: 395
})


In [3]:
print(data.column_names)

['intstruction']


In [3]:
model_path= '/root/autodl-tmp/qwen/Qwen2-7B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
print(tokenizer.pad_token_id)
print(tokenizer.decode(tokenizer.pad_token_id))
print(tokenizer.pad_token)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


151643
<|endoftext|>
<|endoftext|>


In [5]:
def tokenize_function(examples):
    # extract text
    text = examples["intstruction"]

    #tokenize and truncate text
    #保证seq 长度是一样的。
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        f"{text}",
        # return_tensors="np", # return numpy array
        # add_special_tokens=True, # special 会被忽略
        truncation=True,
    )
    # print(text)
    # print(tokenized_inputs)
    # return {
    #     "input_ids": tokenized_inputs['input_ids'] + [tokenizer.pad_token_id],
    #     "attention_mask": tokenized_inputs['attention_mask'] + [1],
    #      "labels": tokenized_inputs['input_ids'] + [tokenizer.pad_token_id]
    # }
    return {
        "input_ids": tokenized_inputs['input_ids'] ,
        "attention_mask": tokenized_inputs['attention_mask'] ,
         "labels": tokenized_inputs['input_ids']
    }

# tokenize training and validation datasets
tokenized_data = data.map(tokenize_function, remove_columns=data.column_names)
print(tokenized_data)
print(tokenized_data[0])
tokenizer.decode(tokenized_data[0]['input_ids'])

Map:   0%|          | 0/395 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 395
})
{'input_ids': [151644, 8948, 198, 56568, 101909, 20316, 85948, 92313, 28946, 425, 10236, 104, 247, 9370, 20002, 11, 220, 103929, 88802, 20412, 104787, 100744, 103936, 151645, 198, 151644, 872, 198, 1, 99727, 100525, 3837, 102056, 30534, 9407, 46944, 100780, 3837, 102762, 50404, 100165, 101036, 11319, 1, 151645, 198, 151644, 77091, 198, 99727, 100525, 73670, 9407, 26288, 109790, 101037, 92313, 28946, 107347, 103322, 103932, 36407, 9407, 18947, 101231, 103794, 99172, 50930, 31251, 58, 109014, 99246, 1457, 109014, 99246, 1457, 109014, 99246, 60, 151645], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [151644, 8948, 198, 56568, 101909, 20316, 85948, 92313, 28946, 425, 10236, 104, 247,

'<|im_start|>system\n你是一个 bilibili 或者 B 站的用户, 你的任务是回复网友的问题<|im_end|>\n<|im_start|>user\n"阿猛，如果你要cos一个角色，你会选择谁呢？"<|im_end|>\n<|im_start|>assistant\n阿猛可以cos大嫂吗 或者缸灿团队来cos个狂飙想看！！[星星眼][星星眼][星星眼]<|im_end|>'

In [6]:
tokenizer.decode(tokenized_data[300]['input_ids'])

'<|im_start|>system\n你是一个 bilibili 或者 B 站的用户, 你的任务是回复网友的问题<|im_end|>\n<|im_start|>user\n"哥们，你在B界西游里混到哪个级别了，有什么特权不？"<|im_end|>\n<|im_start|>assistant\n好家伙盘古级是吧[笑哭]<|im_end|>'

In [7]:
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="cuda",torch_dtype=torch.bfloat16)
model.enable_input_require_grads()
model.dtype

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

torch.bfloat16

In [8]:
# lora config
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
    # 玄学调参！！！
)
print(config)


LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'o_proj', 'k_proj', 'gate_proj', 'q_proj', 'down_proj', 'v_proj', 'up_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)


In [9]:
model = get_peft_model(model, config)
model.print_trainable_parameters()
# 大幅减少 trainable params

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.26434798934534914


In [10]:
# 配置训练参数
args = TrainingArguments(
    output_dir="/root/loraoutput/Qwen2_instruct_lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=10, # 为了快速演示，这里设置10，建议你设置成100
    learning_rate=1e-4,
    save_on_each_node=True,
    # gradient_checkpointing=True,
    load_best_model_at_end=True,
)
# print(args)

In [11]:
print(tokenizer.decode(tokenized_data[200]['input_ids']))
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_data,
    # eval_dataset=tokenized_data["test"],
    # 自动 padding
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


<|im_start|>system
你是一个 bilibili 或者 B 站的用户, 你的任务是回复网友的问题<|im_end|>
<|im_start|>user
"如果你和朋友走向了不同道路，发现彼此不再同步，你会怎么面对这种情况？"<|im_end|>
<|im_start|>assistant
年轻时，你与朋友一起初入社会。可他却因为一次机会与你争吵，甚至丢弃一切，也要抓住机会。你尝试追上去，将他劝回，但你发现你已跟不上他的步伐。这时候你才知道你和他早已不在同一条道路上。你终于放弃，走上自己的道路，却发现本你存在的机会出现在自己面前。机会带着你走向高塔，就在你准备进入高塔时。朋友才从迷雾中走出，你伸出手想把他拉上来，可他早已疲惫不堪，再无爬塔之力。<|im_end|>


In [12]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,3.050000
20,2.400500
30,2.180600
40,2.286300
50,2.111100
60,2.100500
70,1.976100


/root/miniconda3/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /root/autodl-tmp/qwen/Qwen2-7B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /root/autodl-tmp/qwen/Qwen2-7B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:4

TrainOutput(global_step=72, training_loss=2.2968741721577115, metrics={'train_runtime': 94.9651, 'train_samples_per_second': 12.478, 'train_steps_per_second': 0.758, 'total_flos': 7617608433017856.0, 'train_loss': 2.2968741721577115, 'epoch': 2.909090909090909})

In [33]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel
mode_path = '/root/autodl-tmp/qwen/Qwen2-7B-Instruct/'
lora_path = '/root/loraoutput/Qwen2_instruct_lora2/checkpoint-2000' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

prompt = "你是谁？"
inputs = tokenizer.apply_chat_template([{"role": "user", "content": "假设你是皇帝身边的女人--甄嬛。"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)